In [2]:
from ollama import chat
from ollama import ChatResponse

In [2]:

response: ChatResponse = chat(model='phi4', messages=[
  {
    'role': 'user',
    'content': 'Какого цвета небо?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

KeyboardInterrupt: 

In [2]:
!pip install -qU langchain-ollama

In [3]:
from ollama import embed
from ollama import EmbedResponse
from langchain_ollama import OllamaEmbeddings
from IPython.display import display, Markdown, Latex


In [4]:

response: EmbedResponse = embed(model='phi4', input= 'Какого цвета небо?')
print(response['embeddings'])
# or access fields directly from the response object

[[0.016892347, 0.0062055746, 0.009974119, -0.0076360973, -0.019441782, -0.017104601, 0.024603458, -0.028928919, -0.010430361, -0.015230417, -0.008880917, 0.034899615, -0.0009868176, -0.012384443, -0.013325935, 0.0031438125, -0.0069336784, -0.018909132, 0.016121132, -0.0068702046, -0.0147421695, 0.002884516, -0.01334765, 0.0027098518, -0.012841469, 0.0073403157, 0.005192626, 0.009703239, 0.0072893016, -0.00728303, -0.010162966, 0.011609578, -0.0030345437, 0.004588655, 6.2003935e-05, -0.0008190651, -0.024779033, 0.01972959, 0.0013951928, -0.004787706, -0.009651307, -0.024987092, 0.012052034, -0.016236579, -0.00041235433, 0.0029925664, 0.00730462, 0.015266805, -0.022359362, 0.018773047, -0.029719016, -0.0007758416, -0.0172197, 0.002588135, 0.031356093, -0.0014101448, -0.003123805, -0.0020130114, -0.0011964784, -0.0055286814, -0.003033486, -0.00377513, 0.006162683, 0.004687235, -0.0066246516, 0.007891243, -0.004094178, 0.00631557, 0.0005474136, 0.01189265, 0.0009788834, 0.0054882416, 0.005

In [4]:
import requests
import numpy as np
import faiss
import os
from langchain_chroma import Chroma
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Создание базы данных

In [54]:
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text
f = open('essay.txt', 'w')
f.write(text)
f.close()
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

37

In [5]:
def get_chunks_from_text(text, chunk_size = 2048):
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    if(len(chunks) > 1):
        chunks[-2] = chunks[-2] + chunks[-1]
        chunks.pop()
    return chunks

In [6]:
num_chapters = 38
chunks = []
for i in range(1, num_chapters + 1):
    text = ""
    with open(f'./chapters/chapter{i}.tex', 'r') as f:
        text = f.read()
    print(len(text))
    chunks.extend(get_chunks_from_text(text, 16384))
len(chunks)

21365
12946
8044
9589
14721
12330
6663
7409
5915
5297
11028
17966
10902
2789
11267
4523
23586
8382
7971
9995
6880
6417
6308
9777
6519
7145
18035
12872
16519
9473
17236
25274
5302
4372
25815
17268
25280
19034


38

In [5]:
num_chapters = 38
chapters_pages = []
for i in range(1, num_chapters + 1):
    text = ""
    with open(f'./chapters/chapter{i}.tex', 'r') as f:
        text = f.read()
    chapters_pages.append(text)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=16384, 
    chunk_overlap=500,
    length_function=len,
    is_separator_regex=False
)
chunks = text_splitter.create_documents(chapters_pages)

In [29]:
def get_text_embedding(input, output_type = np.array):
    response: EmbedResponse = embed(model="mxbai-embed-large", input=input) # "mxbai-embed-large", "snowflake-arctic-embed2"
    return output_type(response['embeddings'][0]).astype('float32')

In [30]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [354]:
res = faiss.StandardGpuResources()  # use a single GPU

In [167]:
type(text_embeddings[0][0])

numpy.float32

In [343]:
d = text_embeddings.shape[1]

In [327]:
d

4096

In [169]:
faiss.normalize_L2(text_embeddings)

In [170]:
np.linalg.norm(text_embeddings[0])

1.0

In [344]:
index_flat = faiss.IndexFlatL2(d)
index_flat.add(text_embeddings)
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

# DB2

In [6]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

Создат векторную базу данных

In [7]:
vector_store = Chroma.from_documents(
    chunks,
    embeddings,
    collection_name="gosbook_collection",
    persist_directory="./gosbook_collection_db",  # Where to save data locally, remove if not necessary
)

KeyboardInterrupt: 

Загрузить уже сохраненную базу данных

In [11]:
vector_store = Chroma(
    collection_name="gosbook_collection",
    embedding_function = embeddings,
    persist_directory="./gosbook_collection_db",  # Where to save data locally, remove if not necessary
)

In [20]:
def guess_answer_phi(question, model="phi4"):
    user_message = f"""

    Придумай правдоподобный ответ на вопрос.
    Query: {question}
    Answer: 
    """
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    response: ChatResponse = chat(model=model, messages=messages)
    answer = f"""
    {question}
    ---------------------
    {response.message.content}
    """
    return answer

In [102]:
question = "Какие особенности у формулы Тейлора с остаточным членом в форме Пеано?"

In [12]:
question = "Как исследовать функцию на минимум/максимум?"

In [128]:
question = "Вид формулы Тейлора с остаточным членом в форме Пеано"

In [121]:
question = "Критерий Коши сходимости числовой последовательности"

In [149]:
question = "Критерий Коши для функционыльных рядов"

In [129]:
prompt_question = guess_answer_phi(question)

In [13]:
prompt_question = question

In [130]:
prompt_question

'\n    Вид формулы Тейлора с остаточным членом в форме Пеано\n    ---------------------\n    Формула Тейлора с остаточным членом, выраженная в форме Пеано, представляет собой разложение функции в ряд на основе её производных в точке. В традиционной формуле Тейлора остаточный член (остаток) обозначается как Rₙ(x), и он измеряет разницу между фактической функцией и её полиномиальным приближением заданного порядка. В форме Пеано остаточный член представляется в виде:\n\n\\[ R_n(x) = o((x-a)^n) \\]\n\nгде \\( a \\) — точка, вокруг которой производится разложение, а \\( n \\) — порядок ряда. Символ \\( o((x-a)^n) \\) означает, что:\n\n\\[ \\lim_{x \\to a} \\frac{R_n(x)}{(x-a)^n} = 0. \\]\n\nЭто указывает на то, что остаточный член стремится к нулю быстрее любой степени \\( (x-a)^n \\) при \\( x \\to a \\).\n\nТаким образом, форма Пеано для остаточного члена акцентирует внимание на том, что разница между функцией и её полиномиальным приближением становится незначительной гораздо быстрее, чем

In [131]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}
)
res =  retriever.invoke(prompt_question)

In [23]:
res = vector_store.similarity_search_with_score(prompt_question, k=5)

In [132]:
res

[Document(id='02254e9f-e7a4-49b9-87bc-3d4295d98e64', metadata={}, page_content="$$\nr_n(f;x) = o((x-x_0)^n)\n$$\nБудем доказывать последнюю формулу, применяя индукцию. При $n=1$ данное утверждение верно. Действительно, в этом случае функция $f$ дифференцируема в точке $x_0$. Следовательно, согласно лемме \\ref{ch5:lemmlinpribp}\n$$\nf(x) - f(x_0) = f'(x_0) (x-x_0) + o(x-x_0),\\quad x\\to x_0,\n$$\nчто совпадает с утверждением теоремы и с равенством $r_1(f;x)=o(x-x_0)$."),
 Document(id='3d84adcf-43a7-4930-b300-4e237cdf2301', metadata={}, page_content='\\begin{thm}[формула Тейлора с остаточным членом в форме Лагранжа]\nЕсли функция $f(x)$ в некоторой окрестности $O(x_0)$ точки $x_0$ имеет непрерывную производную $n$-го порядка и $f^{(n)}(x)$ дифференцируема в проколотой окрестности $\\overset{\\circ}{O}(x_0)$, то для любого\n$x \\in \\overset{\\circ}{O}(x_0)$ существует $\\xi$, лежащее строго между $x$ и $x_0$ и такое, что справедливо равенство\n\\begin{equation}'),
 Document(id='0a26ce9

In [133]:
retrieved_chunk = [e.page_content for e in res]

In [134]:
retrieved_chunk

["$$\nr_n(f;x) = o((x-x_0)^n)\n$$\nБудем доказывать последнюю формулу, применяя индукцию. При $n=1$ данное утверждение верно. Действительно, в этом случае функция $f$ дифференцируема в точке $x_0$. Следовательно, согласно лемме \\ref{ch5:lemmlinpribp}\n$$\nf(x) - f(x_0) = f'(x_0) (x-x_0) + o(x-x_0),\\quad x\\to x_0,\n$$\nчто совпадает с утверждением теоремы и с равенством $r_1(f;x)=o(x-x_0)$.",
 '\\begin{thm}[формула Тейлора с остаточным членом в форме Лагранжа]\nЕсли функция $f(x)$ в некоторой окрестности $O(x_0)$ точки $x_0$ имеет непрерывную производную $n$-го порядка и $f^{(n)}(x)$ дифференцируема в проколотой окрестности $\\overset{\\circ}{O}(x_0)$, то для любого\n$x \\in \\overset{\\circ}{O}(x_0)$ существует $\\xi$, лежащее строго между $x$ и $x_0$ и такое, что справедливо равенство\n\\begin{equation}',
 'Иногда, чтобы указать, что многочлен Тейлора построен для конкретной функции~$f$ пишут~$P_n(f;x)$,~$r_n(f;x)$ вместо $P_n(x)$,~$r_n(x)$, или даже $P_n(f;x_0;x)$,~$r_n(f;x_0;x)$,

In [218]:
question

'Критерий Коши сходимости числовой последовательности'

In [345]:
question_embeddings = np.array([get_text_embedding(question)])

In [346]:
question_embeddings

array([[-0.01516677,  0.00102726, -0.07758157, ..., -0.00809159,
        -0.03781135,  0.03646936]], dtype=float32)

In [347]:
D, I = gpu_index_flat.search(question_embeddings, k=5) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [348]:
len(retrieved_chunk[0])

17966

In [349]:
D

array([[1.3390608, 1.352026 , 1.3659804, 1.3786051, 1.3801272]],
      dtype=float32)

In [350]:
I

array([[11,  2, 17,  5,  6]])

In [135]:
retrieved_chunk

["$$\nr_n(f;x) = o((x-x_0)^n)\n$$\nБудем доказывать последнюю формулу, применяя индукцию. При $n=1$ данное утверждение верно. Действительно, в этом случае функция $f$ дифференцируема в точке $x_0$. Следовательно, согласно лемме \\ref{ch5:lemmlinpribp}\n$$\nf(x) - f(x_0) = f'(x_0) (x-x_0) + o(x-x_0),\\quad x\\to x_0,\n$$\nчто совпадает с утверждением теоремы и с равенством $r_1(f;x)=o(x-x_0)$.",
 '\\begin{thm}[формула Тейлора с остаточным членом в форме Лагранжа]\nЕсли функция $f(x)$ в некоторой окрестности $O(x_0)$ точки $x_0$ имеет непрерывную производную $n$-го порядка и $f^{(n)}(x)$ дифференцируема в проколотой окрестности $\\overset{\\circ}{O}(x_0)$, то для любого\n$x \\in \\overset{\\circ}{O}(x_0)$ существует $\\xi$, лежащее строго между $x$ и $x_0$ и такое, что справедливо равенство\n\\begin{equation}',
 'Иногда, чтобы указать, что многочлен Тейлора построен для конкретной функции~$f$ пишут~$P_n(f;x)$,~$r_n(f;x)$ вместо $P_n(x)$,~$r_n(x)$, или даже $P_n(f;x_0;x)$,~$r_n(f;x_0;x)$,

In [141]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query. Answer in Russian. Замени в ответе все символы '\\(' и '\\)' на символ '$'.
Query: {question}
Answer:
"""

In [142]:
prompt

'\nContext information is below.\n---------------------\n["$$\\nr_n(f;x) = o((x-x_0)^n)\\n$$\\nБудем доказывать последнюю формулу, применяя индукцию. При $n=1$ данное утверждение верно. Действительно, в этом случае функция $f$ дифференцируема в точке $x_0$. Следовательно, согласно лемме \\\\ref{ch5:lemmlinpribp}\\n$$\\nf(x) - f(x_0) = f\'(x_0) (x-x_0) + o(x-x_0),\\\\quad x\\\\to x_0,\\n$$\\nчто совпадает с утверждением теоремы и с равенством $r_1(f;x)=o(x-x_0)$.", \'\\\\begin{thm}[формула Тейлора с остаточным членом в форме Лагранжа]\\nЕсли функция $f(x)$ в некоторой окрестности $O(x_0)$ точки $x_0$ имеет непрерывную производную $n$-го порядка и $f^{(n)}(x)$ дифференцируема в проколотой окрестности $\\\\overset{\\\\circ}{O}(x_0)$, то для любого\\n$x \\\\in \\\\overset{\\\\circ}{O}(x_0)$ существует $\\\\xi$, лежащее строго между $x$ и $x_0$ и такое, что справедливо равенство\\n\\\\begin{equation}\', \'Иногда, чтобы указать, что многочлен Тейлора построен для конкретной функции~$f$ пишут

In [137]:
def run_phi(user_message, model="phi4"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    response: ChatResponse = chat(model=model, messages=messages)
    return response.message.content

In [143]:
answer_for_prompt = run_phi(prompt)

In [144]:
answer_for_prompt

'Формула Тейлора с остаточным членом в форме Пеано выражает функцию \\( f(x) \\) как сумму её многочлена Тейлора и остаточного члена, который удовлетворяет условию в форме Пеано. Для функции \\( f(x) \\), которая имеет непрерывные производные до порядка \\( n \\) в окрестности точки \\( x_0 \\), формула выглядит следующим образом:\n\n\\[\nf(x) = P_n(f;x) + r_n(f;x),\n\\]\n\nгде многочлен Тейлора \\( P_n(f;x) \\) задается как\n\n\\[\nP_n(f;x) = \\sum_{k=0}^{n} \\frac{1}{k!} f^{(k)}(x_0)(x-x_0)^k,\n\\]\n\nи остаточный член \\( r_n(f;x) \\) удовлетворяет условию\n\n\\[\nr_n(f;x) = o((x-x_0)^n), \\quad x \\to x_0.\n\\]\n\nЭто значит, что при приближении точки \\( x \\) к точке \\( x_0 \\), остаточный член \\( r_n(f;x) \\) стремится к нулю быстрее, чем любая степень \\((x-x_0)^n\\). Это условие выражает поведение разности между функцией и её аппроксимацией многочленом Тейлора.'

In [185]:
import re

pattern = r"(\\\()|(\\\))"
repl = '$'
result_, _ = re.subn(pattern, repl, answer_for_prompt)
pattern = r"\\\[|\\\]"
repl = '$$'
result, _ = re.subn(pattern, repl, result_)


In [186]:
pattern

'\\\\\\[|\\\\\\]'

In [187]:
result

'Формула Тейлора с остаточным членом в форме Пеано выражает функцию $ f(x) $ как сумму её многочлена Тейлора и остаточного члена, который удовлетворяет условию в форме Пеано. Для функции $ f(x) $, которая имеет непрерывные производные до порядка $ n $ в окрестности точки $ x_0 $, формула выглядит следующим образом:\n\n$$\nf(x) = P_n(f;x) + r_n(f;x),\n$$\n\nгде многочлен Тейлора $ P_n(f;x) $ задается как\n\n$$\nP_n(f;x) = \\sum_{k=0}^{n} \\frac{1}{k!} f^{(k)}(x_0)(x-x_0)^k,\n$$\n\nи остаточный член $ r_n(f;x) $ удовлетворяет условию\n\n$$\nr_n(f;x) = o((x-x_0)^n), \\quad x \\to x_0.\n$$\n\nЭто значит, что при приближении точки $ x $ к точке $ x_0 $, остаточный член $ r_n(f;x) $ стремится к нулю быстрее, чем любая степень $(x-x_0)^n$. Это условие выражает поведение разности между функцией и её аппроксимацией многочленом Тейлора.'

In [188]:
display(Markdown(result))

Формула Тейлора с остаточным членом в форме Пеано выражает функцию $ f(x) $ как сумму её многочлена Тейлора и остаточного члена, который удовлетворяет условию в форме Пеано. Для функции $ f(x) $, которая имеет непрерывные производные до порядка $ n $ в окрестности точки $ x_0 $, формула выглядит следующим образом:

$$
f(x) = P_n(f;x) + r_n(f;x),
$$

где многочлен Тейлора $ P_n(f;x) $ задается как

$$
P_n(f;x) = \sum_{k=0}^{n} \frac{1}{k!} f^{(k)}(x_0)(x-x_0)^k,
$$

и остаточный член $ r_n(f;x) $ удовлетворяет условию

$$
r_n(f;x) = o((x-x_0)^n), \quad x \to x_0.
$$

Это значит, что при приближении точки $ x $ к точке $ x_0 $, остаточный член $ r_n(f;x) $ стремится к нулю быстрее, чем любая степень $(x-x_0)^n$. Это условие выражает поведение разности между функцией и её аппроксимацией многочленом Тейлора.